# Utilities

## PushBackCharStream

The `PushBackCharStream` is used by the `Reader` to provide a stream of
characters. It allows for pushing characters back onto the stream while
maintaining line and column info.

In [ ]:
#export
from collections.abc import Iterator

class PushBackCharStream:
    def __init__(self, chars):
        self.iterator = chars if isinstance(chars, Iterator) else iter(chars)
        self.pushed_back = []
        self.line = 0
        self.col = 0
        self.line_history = [0]
        self.__reached_end = False

    def __iter__(self):
        return self

    def __next__(self):
        if self.__reached_end:
            raise StopIteration
            
        if self.pushed_back:
            char = self.pushed_back.pop()
        else:
            try:
                char = next(self.iterator)
            except StopIteration:
                self.__reached_end = True
                return None

        # advanced character position
        if char == '\n':
            self.line += 1
            self.col = 0
        else:
            self.col += 1

        # Save last line history for pushback
        if self.line >= len(self.line_history):
            self.line_history.append(0)
        self.line_history[self.line] = self.col

        return char

    def push_back_single(self, char: str):
        if char is None:
            return
        
        self.pushed_back.append(char)

        # Reverse the position
        if self.col == 0:
            self.line -= 1
            self.col = self.line_history[self.line]
        else:
            self.col -= 1
        
        # Reset EOF since we know we are not there anymore
        self.__reached_end = False
            
    def push_back(self, chars: str):
        if chars is None:
            self.__reached_end = False
            return
        
        for c in reversed(chars):
            self.push_back_single(c)

    @property
    def empty(self):
        return len(self.pushed_back) == 0 and self.__reached_end
    
    def starting_line_col_info(self):
        return (self.line, self.col - 1)
    
    def ending_line_col_info(self):
        return (self.line, self.col)

In [ ]:
# PushBackCharStream
# tests

stream = PushBackCharStream('123456789')
assert list(stream) == list(iter('123456789')) + [None], "Pushback should mimic string Iterator with None at end"

stream = PushBackCharStream('123')
assert next(stream) == '1', "Pushback should return chars in order"
assert next(stream) == '2', "Pushback should return chars in order"
assert next(stream) == '3', "Pushback should return chars in order"

stream = PushBackCharStream('12\n3')
assert stream.col == 0 and stream.line == 0,                   "Keeps track of column and line position"
assert next(stream) and stream.col == 1 and stream.line == 0, "Keeps track of column and line position"
assert next(stream) and stream.col == 2 and stream.line == 0, "Keeps track of column and line position"
assert next(stream) and stream.col == 0 and stream.line == 1, "Keeps track of column and line position"

stream = PushBackCharStream('12\n3')
assert next(stream) == '1' and stream.col == 1 and stream.line == 0
stream.push_back('1')
assert next(stream) == '1' and stream.col == 1 and stream.line == 0,"Can push back characters on to same line"

stream = PushBackCharStream('12\n3')
next(stream);next(stream);next(stream)
assert next(stream) == '3' and stream.col == 1 and stream.line == 1, "Can pushback characters spanning lines"
stream.push_back('3'); 
assert stream.col == 0 and stream.line == 1,                         "Can pushback characters spanning lines"
stream.push_back('\n')
assert stream.col == 2 and stream.line == 0,                         "Can pushback characters spanning lines"

stream = PushBackCharStream('12')
next(stream)
assert stream.starting_line_col_info() == (0, 0), 'Starting line col info is previous character position'
assert stream.ending_line_col_info() == (0, 1),   'Ending line col info is next character position'

In [ ]:
#export
from collections import namedtuple

LineColInfo = namedtuple('LineColInfo', ['start_line', 'start_col', 'end_line', 'end_col'])

# Flask Tools
Utilities to help running flask apps in jupyter notebooks

In [ ]:
# export

import threading
from pathlib import Path

from flask import Flask, abort, g, request, send_from_directory, Blueprint
from flask_cors import cross_origin
import waitress

class TruthyMap(dict):
    '''Allows a clean exit from the waitress server run loop

    I really haven't looked into this in too much detail but
    server.run essentially loops with the following expression:

        `while map:`
    
    and if no map is passed into `create_server` then an empty
    dict is used. So if this map can just express a truthy value
    that we control then we can exit the run loop
    '''
    def __init__(self, *args, **kwargs):
        self.truthy = True
        super().__init__(*args, **kwargs)

    def __bool__(self):
        return self.truthy

def add_nocache_headers(r):
    """
    Add headers to both force latest IE rendering engine or Chrome Frame,
    and also to cache the rendered page for 10 minutes.
    """
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r
    
class JupyterFlask:
    '''An object that is meant to mirror a Flask object with the added
    benefit that it can be used in a Jupyter notebook. Each time a new route
    is added via `route` an existing server will be shutdown and a new app
    and server will be constructed.

    Waitress is used as the server. The reasons for waitress is as follows:
        1. I normally use waitress
        2. I was able to get waitress to cleanly stop (albeit in a hacky manner
           as its really not intended to be stopped except by a process exiting)

    Handlers can be overridden by using the same route `rule`. The only
    restriction on creating a route is that any distinct route (i.e. unique
    rule) requires that that the handler's function name must be unique across
    all other active routes. This is something im pretty sure is the same in
    creating a normal Flask app and so I decided to keep it the same.

    Note: The waitress server is created and run each time a route is
    created. So there is no need to start the app.

    Note2: `cross_origin` wraps each handler. This is required for running out of 
    a jupyter notebook as it undoubtedly will be called cross origin. It looks
    like you can also use `cross_origin` on the handler definition in your 
    notebook without issue.
    '''
    def __init__(self, name, *args, static_url_path=None, static_folder='static', **kwargs):
        self.name = name
        self.port = kwargs.pop('port', 8845)
        self.args = [name] + list(args)
        self.kwargs = kwargs
        self.handlers = {}
        self.blueprints = {}
        
        # Need to add this ourselves for cors support
        if static_url_path is not None:
            def static_handler(path):
                return send_from_directory(Path(static_folder).resolve(), path)
            self.handlers[f"{static_url_path}/<path:path>"] = ({}, static_handler)
            
            # deregister the default static view for flask
            kwargs['static_folder'] = None

        self.app = None
        self.server = None 
        self.thread = None 
        self.map = None

    def restart_server(self):
        '''Stops existing running server if exists and reconstructs app and
        starts a waitress server'''
        if self.server is not None:
            # This is all pretty implementation specific so maybe won't work
            # for all past or future versions of waitress
            self.server.close()
            self.server.task_dispatcher.set_thread_count(0)   # this will close the waitress threads
            self.map.truthy = False                           # this will close our thread

            # This allows us to know if we are cleanly exiting waitress. If we
            # are not then we will hang here
            waitress_threads = [thread
                                for thread in threading.enumerate()
                                if thread.name.startswith('waitress')]
            for thread in waitress_threads:
                thread.join(timeout=5.0)
                assert not thread.is_alive(), 'Waitress thread is still alive'

        self.app = Flask(*self.args, **self.kwargs)
        
        self.app.after_request(add_nocache_headers)
        
        for rule, options_and_handler in self.handlers.items():
            options, handler = options_and_handler
            self.app.route(rule, **options)(cross_origin()(handler))
            
        for prefix, blueprint_and_args in self.blueprints.items():
            blueprint, args, kwargs = blueprint_and_args
            self.app.register_blueprint(blueprint.blueprint, *args, **kwargs)

        self.map = TruthyMap()
        self.server = waitress.create_server(self.app, port=self.port, map=self.map)
        self.thread = threading.Thread(name='waitress-run-loop', target=lambda: self.server.run())
        self.thread.start()

    def route(self, rule, **options):
        def wrapper(f):
            self.handlers[rule] = (options, f)
            self.restart_server()
        return wrapper
    
    def register_blueprint(self, blueprint, *args, **kwargs):
        url_prefix = kwargs.pop('url_prefix', '/')
        kwargs['url_prefix'] = url_prefix
        self.blueprints[url_prefix] = (blueprint, args, kwargs)
        self.restart_server()
 

class JupyterBlueprint:
    '''Simple wrapper that will restart the app when a route is added'''
    def __init__(self, app, blueprint):
        self.app = app
        self.blueprint = blueprint
        self.handlers = {}

    def route(self, rule, **options):
        def wrapper(f):
            self.handlers[rule] = (options, f)
            
            # create a new blueprint and add all routes
            bp = self.blueprint
            self.blueprint = Blueprint(bp.name, 
                                       bp.import_name,
                                       bp.static_folder,
                                       bp.static_url_path,
                                       bp.template_folder,
                                       bp.url_prefix,
                                       bp.subdomain,
                                       None, # bp.url_defaults,
                                       bp.root_path,
                                       bp.cli_group)
            
            self.blueprint.after_request(add_nocache_headers)

            for r, option_and_handler in self.handlers.items():
                opts, handler = option_and_handler
                self.blueprint.route(r, **opts)(cross_origin()(handler))
                
            self.app.restart_server()
        return wrapper
        
        

In [ ]:
app = JupyterFlask(name='test', port=8845, static_url_path='/static', static_folder='public')
bp = Blueprint('test', __name__)
bp2 = JupyterBlueprint(app, bp)

app.register_blueprint(bp2, url_prefix='/api')



In [ ]:
@bp2.route('/hello')
def hello():
    print(request.headers)
    header = request.headers['accept-encoding']
    return b'HELLO WORLD ' + header.encode('utf-8')

In [ ]:
import http.client

conn = http.client.HTTPConnection("localhost:8845")
conn.request("GET", "/api/hello")
res = conn.getresponse()

print(res.status, res.reason)
print(res.read())

In [ ]:
@bp2.route('/hello4')
@cross_origin()
def hello4():
    return b'HELLO WORLD 4'

In [ ]:

conn = http.client.HTTPConnection("localhost:8845")
conn.request("GET", "/api/hello4")
res = conn.getresponse()

print(res.status, res.reason)
print(res.read())

In [ ]:
for t in threading.enumerate():
    print(t.name)